In [ ]:
import os
import sys

# change working directory
current_dir = os.getcwd() 
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
#We need to be in the main directory that contains tests, models, etc folders

In [ ]:
from trading_env_class import stock

In [ ]:
# Create a stock instance with a sample ticker
test_stock = stock("^GSPC")

# Define a test date and price
test_date = price_history["^GSPC"].index[0]  # Use the first date in price_history
test_price = price_history["^GSPC"][test_date]

In [ ]:
# Perform a buy operation
test_stock.sell(test_date, 10)  # Buy 10 shares
print(test_stock.immediate_profit)

test_date2 = price_history["^GSPC"].index[1]  # Use the first date in price_history
test_stock.buy(test_date2, 5)  # Sell 5 shares
expected_profit = 5 * (price_history["^GSPC"][test_date2]-test_price)  # Expected profit from selling 5 shares
print(test_stock.immediate_profit, expected_profit)

test_date21 = price_history["^GSPC"].index[2]  # Use the first date in price_history
test_stock.sell(test_date21, 5)  # Sell 5 shares
expected_profit = 0
print(test_stock.immediate_profit, expected_profit)

test_date3 = price_history["^GSPC"].index[3]  # Use the first date in price_history
test_stock.buy(test_date3, 5)  # Sell 5 shares
expected_profit = 5 * (price_history["^GSPC"][test_date3]-test_price)  # Expected profit from selling 5 shares
print(test_stock.immediate_profit, expected_profit)

print(test_stock.realized_profit)

date5=price_history["^GSPC"].index[5]
test_stock.close(price_history["^GSPC"].index[5])
expected_profit = 5 * (price_history["^GSPC"][date5]-price_history["^GSPC"][test_date21])
print(test_stock.immediate_profit, expected_profit)

print(test_stock.realized_profit)
print("Test passed: .immediate_profit is working as expected.")